In [10]:
import json
import pymysql


In [11]:
with open("product_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

conn = pymysql.connect( host="127.0.0.1", port=3306, db="cosmetic_product",
                        user="humanda", passwd="humanda")
                               
cursor = conn.cursor()

# Product 테이블 데이터 삽입
for product in data:
    category = product["category"]
    product_name = product["product_name"]
    url = product["url"]

    sql_product = """INSERT INTO product (category, url, product_name)
                     VALUES (%s, %s, %s)"""
    cursor.execute(sql_product, (category, url, product_name))                     
    product_id = cursor.lastrowid # 삽입된 제품의 ID 가져오기

    # Ingredient 테이블 데이터 삽입
    for ingredient_data in product["ingredients"]:
        for ingredient_name, count in ingredient_data.items():
            sql_ingredient = """INSERT INTO ingredient (product_no, ingredient, count)
                                VALUES (%s, %s, %s)"""
            cursor.execute(sql_ingredient, (product_id, ingredient_name, count))

conn.commit()

cursor.close()
conn.close()

In [15]:
conn = pymysql.connect( host="127.0.0.1", port=3306, db="cosmetic_product",
                        user="humanda", passwd="humanda")
                               
cursor = conn.cursor()

# 전체 제품 개수 확인
cursor.execute("SELECT COUNT(*) FROM product")
total_products = cursor.fetchone()[0]
print(f"제품 개수: {total_products}")

# 전체 성분 개수 조회
cursor.execute("SELECT COUNT(*) FROM ingredient")
total_ingredients = cursor.fetchone()[0]
print(f"성분 개수: {total_ingredients}")

# 제품별 성분 개수 확인
cursor.execute("""
SELECT product_no, COUNT(*) AS ingredient_count
FROM ingredient
GROUP BY product_no
""")
ingredients_check = cursor.fetchall()[:5]
print(f"제품별 성분 개수: {ingredients_check}")

cursor.close()
conn.close()

제품 개수: 21197
성분 개수: 46332
제품별 성분 개수: ((1, 1), (2, 1), (3, 4), (5, 2), (6, 3))
